In [9]:
!nvidia-smi

Fri Mar 17 08:37:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    28W /  70W |    573MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7f9093678220>)

In [11]:
torch.cuda.device_count()

1

In [12]:
"""以下两个函数保证了当请求GPU不存在的情况下仍然可以运行代码"""
def try_gpu(i=0):
  """如果存在，则返回gpu(i)，否则返回cpu()"""
  if torch.cuda.device_count() >= i+1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus():
  """返回所有可用gpu，如果没有GPU， 返回[cpu(),]"""
  devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [13]:
"""查询张量所在设备"""
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [14]:
"""存储在GPU上"""
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [15]:
"""在第二个GPU上创建一个随机张量"""
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.2220, 0.5823, 0.6948],
        [0.8425, 0.8729, 0.8235]], device='cuda:0')

In [16]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [17]:
Y + Z

tensor([[1.2220, 1.5823, 1.6948],
        [1.8425, 1.8729, 1.8235]], device='cuda:0')

In [18]:
Z.cuda(0) is Z

True

In [19]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device = try_gpu())

net(X)

tensor([[-0.8331],
        [-0.8331]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [20]:
net[0].weight.data.device

device(type='cuda', index=0)